In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import polars as pl
import re
import nltk

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = set(stopwords.words("english"))
def fast_clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+|[^a-zA-Z\s]", "", text)
    tokens = re.findall(r'\b\w+\b', text)
    return " ".join([t for t in tokens if t not in stop_words])

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
from datasets import load_dataset
dataset = load_dataset("artem9k/ai-text-detection-pile")["train"]
seen, texts, labels = set(), [], []
for sample in dataset:
    if sample["text"] and sample["text"].strip() and sample["text"] not in seen:
        seen.add(sample["text"])
        texts.append(sample["text"])
        labels.append(1 if sample["source"] == "ai" else 0)

df = pl.DataFrame({"text": texts, "label": labels})
df = df.with_columns(pl.Series("cleaned_text", [fast_clean(t) for t in df["text"]]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00007-bc5952582e004d67.parquet:   0%|          | 0.00/758M [00:00<?, ?B/s]

(…)-00001-of-00007-71c80017bc45f30d.parquet:   0%|          | 0.00/318M [00:00<?, ?B/s]

(…)-00002-of-00007-ee2d43f396e78fbc.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

(…)-00003-of-00007-529931154b42b51d.parquet:   0%|          | 0.00/137M [00:00<?, ?B/s]

(…)-00004-of-00007-b269dc49374a2c0b.parquet:   0%|          | 0.00/137M [00:00<?, ?B/s]

(…)-00005-of-00007-3dce5e05ddbad789.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

(…)-00006-of-00007-3d8a471ba0cf1c8d.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1392522 [00:00<?, ? examples/s]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df["cleaned_text"].to_list(), df["label"].to_list(),
    test_size=0.2, random_state=42
)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
bow_vectorizer = CountVectorizer(max_features=10000, stop_words='english')
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_val_bow = bow_vectorizer.transform(X_val)


In [ ]:
svd = TruncatedSVD(n_components=1000)
X_train_reduced = svd.fit_transform(X_train_bow)
X_val_reduced = svd.transform(X_val_bow)

In [ ]:
X_train_seq = X_train_reduced.reshape(-1, 10, 100)
X_val_seq = X_val_reduced.reshape(-1, 10, 100)

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout

def lstm_model():
    return Sequential([
        LSTM(64, input_shape=(10, 100)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])


In [ ]:
def bilstm_model():
    return Sequential([
        Bidirectional(LSTM(64), input_shape=(10, 100)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
def gru_model():
    return Sequential([
        GRU(64, input_shape=(10, 100)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

def train_and_evaluate(model, name):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    model.fit(
        X_train_seq, y_train,
        validation_data=(X_val_seq, y_val),
        epochs=5,
        batch_size=64,
        callbacks=[EarlyStopping(patience=2, restore_best_weights=True)],
        verbose=1
    )

    y_pred = (model.predict(X_val_seq) > 0.5).astype("int32")
    print(f"\n{name} Classification Report:")
    print(classification_report(y_val, y_pred))

In [ ]:
train_and_evaluate(lstm_model(), "LSTM (BoW)")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,353 (173.25 KB)

 Trainable params: 44,353 (173.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 95s 5ms/step - accuracy: 0.8405 - loss: 0.3245 - val_accuracy: 0.8759 - val_loss: 0.2616
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8788 - loss: 0.2565 - val_accuracy: 0.8842 - val_loss: 0.2482
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8877 - loss: 0.2412 - val_accuracy: 0.8843 - val_loss: 0.2455
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8915 - loss: 0.2344 - val_accuracy: 0.8872 - val_loss: 0.2411
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8952 - loss: 0.2286 - val_accuracy: 0.8896 - val_loss: 0.2390
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step

LSTM (BoW) Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.93    204218
           1       0.81      0.76      0.78     72951

    accuracy                           0.89    277169
   macro avg       0.86      0.85      0.86

In [ ]:
train_and_evaluate(bilstm_model(), "BiLSTM (BoW)")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,641 (346.25 KB)

 Trainable params: 88,641 (346.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 114s 6ms/step - accuracy: 0.8480 - loss: 0.3111 - val_accuracy: 0.8828 - val_loss: 0.2501
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 111s 6ms/step - accuracy: 0.8842 - loss: 0.2466 - val_accuracy: 0.8880 - val_loss: 0.2399
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 111s 6ms/step - accuracy: 0.8925 - loss: 0.2322 - val_accuracy: 0.8903 - val_loss: 0.2364
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 110s 6ms/step - accuracy: 0.8987 - loss: 0.2217 - val_accuracy: 0.8925 - val_loss: 0.2328
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 111s 6ms/step - accuracy: 0.9022 - loss: 0.2153 - val_accuracy: 0.8926 - val_loss: 0.2328
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

BiLSTM (BoW) Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93    204218
           1       0.80      0.79      0.79     72951

    accuracy                           0.89    277169
   macro avg       0.86      0.86   

In [ ]:
train_and_evaluate(gru_model(), "GRU (BoW)")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        31,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,985 (132.75 KB)

 Trainable params: 33,985 (132.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 92s 5ms/step - accuracy: 0.8372 - loss: 0.3302 - val_accuracy: 0.8763 - val_loss: 0.2595
Epoch 2/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8794 - loss: 0.2549 - val_accuracy: 0.8842 - val_loss: 0.2467
Epoch 3/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8863 - loss: 0.2431 - val_accuracy: 0.8860 - val_loss: 0.2443
Epoch 4/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8899 - loss: 0.2371 - val_accuracy: 0.8879 - val_loss: 0.2401
Epoch 5/5
17324/17324 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - accuracy: 0.8926 - loss: 0.2330 - val_accuracy: 0.8885 - val_loss: 0.2402
8662/8662 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step

GRU (BoW) Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92    204218
           1       0.78      0.79      0.79     72951

    accuracy                           0.89    277169
   macro avg       0.85      0.86      0.86 